### <center> Tel Aviv University
## <center>**Digital Sciences for High-Tech**
# <center> **Introduction to Machine Learning - Final Project**
#### <center> <i> Spring 2024
    
### <center> Group 11
### <center> Niv Noyman 
### <center> Tom Pashinsky <br>
<br>

#### In this project, we will build a model that predicts if a person will get hired for a job or not.

### **Import libraries**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,FunctionTransformer, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, RocCurveDisplay, accuracy_score, roc_auc_score, confusion_matrix, classification_report,ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.inspection import permutation_importance
import os

### Loading the data

In [ ]:
pd.set_option('display.max_columns',None)
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df_copy = train_df.copy()
test_df_copy = test_df.copy()

<br>
<br>
<br>
<br>
<br>

# **Part A - Exploration**
### First, we will analyze the "train" and "test" datasets: <br>
### 1. **Size:**
How many rows and columns each dataset has: 

In [ ]:
print(f'The size of the training dataset is: {train_df_copy.shape}')
print(f'The size of the test dataset is: {test_df_copy.shape}')

It seems that the "train" dataset is bigger than the "test" dataset.

### 2. **"ID" unique values:**
We will check if all the ID's in the "ID" column appear exactly once: 

In [ ]:
train_df_copy.ID.is_unique

In [ ]:
test_df_copy.ID.is_unique

It seems that all the ID's appear exactly once. Now we can be sure that information doesn't appear multiple times in the datasets.

### 3. **Missing values:**
We will check how many missing values there are in each column (except the "ID" column):

In [ ]:
# Dropping the 'ID' column

train_df_no_id = train_df_copy.drop(columns=['ID'])
test_df_no_id = test_df_copy.drop(columns=['ID'])

# Count the number of NaN values in each column of the dataset.

def count_nans(df, dataset_name):
    nan_counts = df.isna().sum()
    print(f'NaN Counts in {dataset_name}:\n{nan_counts}\n')

count_nans(train_df_no_id, 'Training Dataset')

We can see that the number of missing values in **"stack_experience"** is significantly higher than the other columns. **It doesn't necessarily mean that there 14,042 NaNs, but it is possible that some of the cells are actually 0, which means that a person doesn't know any technologies at all.** <br>
In addition, there are no missing values in the "label" column, so we can use this column for training the models. <br> This problem will be resolved in the preprocessing stage.

### 4. **Categorical variables:**

In [ ]:
# Identify categorical columns by checking their data types

categorical_columns = train_df_copy.select_dtypes(include=['object', 'category']).columns

# Count the number of categorical columns

num_categorical_columns = len(categorical_columns)

print(f"Number of categorical columns: {num_categorical_columns}")

The "train" dataset has 17 columns, 10 of them do not contain numerical values.<br> In some cases each cell contains one of 2 values (for example the "disability" column), in some cases each cell contains one of more than 2 values (for example the "education" column), and there is even a column containing several words in each cell (the "stack_experience" column). <br> We will expand on the way to deal with the problem in the preprocessing stage.

### 5. **Statistics and Graphical Representation:**
<br>

### Histograms
Down here, we plotted histograms for the numeric features: 

In [ ]:
# Plot histograms for numeric features in the dataset.

def plot_histograms(df, dataset_name):
    numeric_cols = df.select_dtypes(include=np.number).columns
    df[numeric_cols].hist(bins=30, figsize=(15, 10))
    plt.suptitle(f'Histograms of Numeric Features - {dataset_name}')
    plt.show()

plot_histograms(train_df_no_id, 'Training Dataset')

- **"years_of_experience" -** it seems that most of the people in the "train" dataset (around 6500) have a little less than 10 years of experience, and as the years of experience increase, the amount of people decreases. In general, we can see in the plot that there is a pattern - the graph increases and than drops, and right after it drops it increases again but the peak point is lower than the previous one.
- **"A" -** it seems that unlike other features, the range of values in feature "A" is very small.
- **"D" -** like the feature "A", it seems that the range of values is very small.
- **"prev_salary" -** it seems that most of the people in the "train" dataset (around 3800) have earned on there previous job a salary of approximately 12500, and as the salary increase, the amount of people decreases. The pattern reminds the one in the "years_of_experience", and this connection makes sense: the higher of experience you have, the higher the salary you earn.
- **"B" -** the meaning of this column is not clear yet, but its graph acts similar like the graphs of "years_of_experience" and "prev_salary". It might indicate a connection between these features, but more exploration is needed.
- **"label" -** there are more people who got hired to the job (label '1', close to 30000) and than those who didn't (label '0', a little higher than 25000). <br>

### Distributions <br>
Here we will plot the distributions of the numeric features using a technique we didn't see in class - **KDE**. <br>
This technique helps to estimate the probability density function of a random variable. It creates a smooth curve from discretely sampled data that reflects the underlying density distribution.

In [ ]:
# Plot distribution plots for numeric features in the dataset.

def plot_distributions(df, dataset_name):
    numeric_cols = df.select_dtypes(include=np.number).columns
    plt.figure(figsize=(15, 10))
    for i, col in enumerate(numeric_cols, 1):
        plt.subplot(len(numeric_cols) // 3 + 1, 3, i)
        sns.histplot(df[col], kde=True)
        plt.title(f'Distribution of Feature "{col}" {dataset_name}')
    plt.tight_layout()
    plt.show()

plot_distributions(train_df_no_id, 'Training Dataset')

According to the plots, the features "A" and "D" are distributing normally. <br>
The distribution of "years_of_experience", "B" and "prev_salary" looks similar but not exactly the same. **In particular, the distribution of "B" is more similar to "years_of_experience" than "prev_salary".** <br>

### Statistics for numeric features
Here we will check some statistical values of the numeric features (Mean, std, min / max values etc.):

In [ ]:
# Display statistical summary of numeric features in the dataset.

def display_numeric_stats(df, dataset_name):
    numeric_cols = df.select_dtypes(include=np.number).columns
    print(f'Numeric Features Statistics for {dataset_name}:\n')
    return df[numeric_cols].describe().transpose()

display_numeric_stats(train_df_no_id, 'Training Dataset')

- **Count -** the numeric features (expect for 'label') have less than 55462 values due to empty cells (NaNs). <br>

- **Min / Max -** if we compare this information with the histograms above, we can see that there are outliers in some of the features: <br>
    - "A" - its minimum value is -21.63 and its maximum is at 55.495, while according to the histogram the minimum is approximately -13 and the maximum reaches to 30.
    - "B" - its maximum value is 50, while according to the histogram the maximum is approximately 42.
    - "D" - its maximum value is 184.15, while according to the histogram the maximum is approximately 183. <br>
    
- Another interesting point is that **all the values statistics of "B" are smaller or equal to "years_of_experience"**. This also might indicate that there is a connection these features. <br> Soon we will create a correlation map so we can check if there is a real connection between them.  

### Statistics for categorial features
Here we will count how many values there are in each column:

In [ ]:
# Display value counts for categorial features in the DataFrame.
    
def display_categorial_stats(df, dataset_name):
    print(f'Categorial Features Statistics for {dataset_name}:\n')
    categorial_cols = df.select_dtypes(include='object').columns
    for col in categorial_cols:
        print(f'Column: {col}')
        print()
        print(df[col].value_counts())
        print()

display_categorial_stats(train_df_no_id, 'Training Dataset')

It looks like the majority of the people in the "train" dataset have worked at least once job. <br> Moreover, most of them are young, men, developers without disabilities or mental issues. Most of them have a BA / Bsc degree, live in western countries (such as the USA, Germany, the United Kingdom etc.) and top 3 popular programming languages are Python, C++ and Git.

### Correlation
In order to see if there are correlations between the features, we will convert the categorial features to numerical ones (the conversion is only for creating the correlation map, we will convert the categorial features in a different way for building the models later on in the notebook) and then create a correlation map:

In [ ]:
# Encode non-numeric columns in the dataset, in order to be able to create a correlation matrix for the analysis.

def replace_words_with_count(df, column_name):
    df[column_name] = df[column_name].apply(lambda cell: len(cell.split(';')) if pd.notna(cell) else 0)
    return df        
def encode_non_numeric_columns(df):
    non_numeric_columns = df.select_dtypes(include=['object']).columns
    label_encoders = {col: LabelEncoder() for col in non_numeric_columns}
    for col in non_numeric_columns:
        df[col] = label_encoders[col].fit_transform(df[col].astype(str))
    return df
train_df_no_id = replace_words_with_count(train_df_no_id,'stack_experience')
train_encoded = encode_non_numeric_columns(train_df_no_id)

# Plot the correlation matrix for numeric features in the dataset.

def plot_correlation_matrix(df, dataset_name):
    numeric_cols = df.select_dtypes(include=np.number).columns
    corr_matrix = df[numeric_cols].corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title(f'Correlation Matrix - {dataset_name}')
    plt.show()
    
plot_correlation_matrix(train_encoded, 'Train Dataset')

According to the map: <br>
- **Most of the features don't have correlations with other features.**
<br>
- **There are few low-medium negative correlations, for example:**
    - "prev_salary" and "age_group" have a negative correlation of -0.26 . It makes sense, because when we converted the categorial features to numeric, it classified "old" as '0' and "young" as '1'. So the more money a person earns, the more likely he will be classified as '0' (old). <br>
<br>    
- **There are few medium-strong negative correlations, for example:**
    - "years_of_experience" and "age_group" have a negative correlation of -0.53 . It makes sense that the more experience a person has, the more likely he will be classified as '0' (old).
    - "B" and "age_group" have a negative correlation of -0.54 . The number is very close to the one mentioned above. <br>
    <br>
- **There are few low-medium positive correlations, for example:**
    - "prev_salary" and "years_of_experience" have a positive correlation of 0.39 . It makes sense that the higher the previous salary of a person is, the more experience he has.
    - "prev_salary" and "B" have a positive correlation of 0.39 . The number is very close to the one mentioned above.
    - "D" and "label" have a positive correlation of 0.41 . We are not sure what "D" means, maybe it's kind of a performance measure that affects the chances of being hired.
    - "stack_experience" and "label" have a positive correlation of 0.36 . It seems that knowing a lot of different technologies increases the chances to get hired for a job. <br>
    <br>
- **There is a strong positive correlation of 0.9 between the features "B" and "years_of_experience"**. It fits the conclusions we made from the histograms, distributions and statistics above. We will address this relationship in the preprocessing step.

## **Here, we will repeat the exploration stage, this time with the "test" dataset**:

In [ ]:
count_nans(test_df_no_id, 'Test Dataset')

In [ ]:
plot_histograms(test_df_no_id, 'Test Dataset')

In [ ]:
plot_distributions(test_df_no_id, 'Test Dataset')

In [ ]:
display_numeric_stats(test_df_no_id, 'Test Dataset')

In [ ]:
display_categorial_stats(test_df_no_id, 'Test Dataset')

In [ ]:
test_encoded = encode_non_numeric_columns(test_df_no_id.copy())
plot_correlation_matrix(test_encoded, 'Test Dataset')

We can see that although the "test" dataset is smaller, its behavior is very similar to the behavior of the "train" dataset.

<br>
<br>
<br>
<br>
<br>

# **Part B - Preprocessing**

In this section we will detail issues that arose during the exploration stage. First we implemented the solutions on the train file, and at the end of the notebook we applied everything on the test file. **We note that all the detailed information below refers to the data from the train file only:**

### 1. **Outliers:**
As we stated in the exploration stage, some of the features have a small number of extreme values. <br>
Therefore, we built a function that removes rows with values smaller than the 0.01 percentile and greater than the 0.99 percentile. <br>
**This step is not supposed to drastically change the dataset and its behaviors.**

In [ ]:
def cap_outliers_percentiles_multi(df, columns, lower_percentile=0.01, upper_percentile=0.99):
    # Iterate over each specified column in the DataFrame
    for column in columns:
        # Calculate the lower and upper bounds based on the specified percentiles
        lower_bound = df[column].quantile(lower_percentile)
        upper_bound = df[column].quantile(upper_percentile)
        
        # Apply the capping: replace values below the lower bound with the lower bound,
        # and values above the upper bound with the upper bound
        df[column] = df[column].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))
    
    # Return the modified DataFrame with capped outliers
    return df

In [ ]:
# Apply the function to relevant columns
numeric_columns = ['A', 'B', 'D']

train_df_copy = cap_outliers_percentiles_multi(train_df_copy, numeric_columns)
train_df_copy

### 2. **Creating new features:**
**Our working assumption is that when considering whether a candidate will be hired, there may be some importance to his knowledge of different technologies and programming languages.** So we will check how many technologies there are in the training dataset, and if there is a correlation between knowing certain programming languages and accepting the candidate for a job:

In [ ]:
def process_stack_experience(train_df, label_column, top_n=20):
    # Split the 'stack_experience' column by ';' and flatten the list
    technologies = train_df['stack_experience'].dropna().str.split(';').tolist()
    flattened_technologies = [tech for sublist in technologies for tech in sublist]

    # Count the occurrences of each technology
    technology_counts = Counter(flattened_technologies)

    # Count the number of unique technologies
    unique_technologies_count = len(technology_counts)

    # Create dummy variables for 'stack_experience'
    stack_experience_dummies = train_df['stack_experience'].str.get_dummies(sep=';')

    # Calculate the correlation matrix between the top 20 technologies and the class '1' in label
    correlation_with_label = stack_experience_dummies.corrwith(train_df[label_column]).sort_values(ascending=False)

    # Select the top N most correlated technologies with the label
    top_technologies = correlation_with_label.head(top_n).index

    # Add the top N most correlated technologies as new columns to the original DataFrame
    train_df = pd.concat([train_df, stack_experience_dummies[top_technologies]], axis=1)
    
    return train_df, unique_technologies_count, top_technologies, correlation_with_label

In [ ]:
train_df_copy, unique_technologies_count, top_technologies, correlation_with_label = process_stack_experience(train_df_copy, 'label')
print(f'There are {unique_technologies_count} different technologies in the dataset.')
print()
print(f'These are the top 20 techs correlated with class 1 in the label column:\n{correlation_with_label.head(20)}')

It seems that there is a significant correlation between proficiency in technologies and getting hired, so we will add the technologies as columns that would receive values of 1 or 0, for technology control or lack of control, respectively. <br>
**We did it in order to understand if the new features could provide us with additional information that would help achieve higher results in running the models.**


Now, we will add the top 20 technologies to the Test dataset. <br> 

**Important note:** since there are no labels in the test dataset, and the "process_stack_experience" function relies on the labels, we simply added to it the technologies that came out after running the function on the training dataset.

In [ ]:
def add_top_technologies_to_test_df(test_df, top_technologies):
    # Create dummy variables for 'stack_experience' in test_df
    stack_experience_dummies_test = test_df['stack_experience'].str.get_dummies(sep=';')
    
    # Select only the top technologies columns
    test_top_tech_dummies = stack_experience_dummies_test.reindex(columns=top_technologies, fill_value=0)
    
    # Add the top technologies as new columns to the original test DataFrame
    test_df = pd.concat([test_df, test_top_tech_dummies], axis=1)
    
    return test_df

In [ ]:
test_df_copy = add_top_technologies_to_test_df(test_df_copy, top_technologies)
test_df_copy

### 3. **Completing missing values:**
We will implement 2 methods to deal with the problem:
- **Connections between features** - there are several features with connections between them:
   - **"B" and "years_of_experience"** - as we saw in the exploration stage, there is a strong positive correlation between this features. In the vast majority of rows, the "B" column values are less than or equal to the "years_of_experience" column values. **It seems that column "B" represents the number of years of experience in the current job.** Therefore, every time we reach a missing cell in one of the columns, we will complete it by choosing the maximum result from two options: the value of the second cell minus the difference of the averages of the columns, or 0 when the value of the second cell minus the difference of the averages of the columns results in a negative number. 

  - **"is_dev" and "stack_experience"** - fill in missing values in the "is_dev" column according to the following rule: if there is a value in the "stack_experience" row that is greater than 0, the value in the "is_dev" row will be developer, otherwise non-developer. <br>
  **The logic behind this course of action is that a person who has stack experience is most likely a software developer.**

  - **"age_group" and "years_of_experience"** - fill in missing values in the "age_group" column according to the following rule: a person with over 20 years of experience will be defined as old, otherwise he will be defined as young.


  - **"worked_in_the_past" and "years_of_experience"** - fill in missing values in the "worked_in_the_past" column according to the following rule: if the value in the "years_of_experience" column is greater than 0, the value in the "worked_in_the_past" column is True, otherwise it is False. <br>
  It makes sense that if someone has experience in working, he must have worked somewhere. <br><br>
  
- **Proportion** - we decided to fill the missing slots in each column (apart from the columns "is_dev", "age_group", "worked_in_the_past") with values that would preserve the original data ratio. **We chose this way because we did not want arbitrary data filling to affect the distribution of features.**  

In [ ]:
 def fill_b_and_experience(df, b_col, exp_col):
    # Calculate the mean values for each column
    b_mean = df[b_col].mean()
    exp_mean = df[exp_col].mean()
    
    # Define a function to fill NaN values based on the given logic
    def fill_b(row):
        if pd.isna(row[b_col]):
            if pd.isna(row[exp_col]):
                # Both values are NaN, use the mean values
                return b_mean
            else:
                # If years_of_experience is not NaN, use max(years_of_experience - 5, 0)
                return max(row[exp_col] - 5, 0)
        else:
            # If B is not NaN, return its value
            return row[b_col]

    def fill_exp(row):
        if pd.isna(row[exp_col]):
            if pd.isna(row[b_col]):
                # Both values are NaN, use the mean values
                return exp_mean
            else:
                # If B is not NaN, use B + 5
                return row[b_col] + 5
        else:
            # If years_of_experience is not NaN, return its value
            return row[exp_col]

    # Apply the functions to each row in the DataFrame
    df[b_col] = df.apply(lambda row: fill_b(row), axis=1)
    df[exp_col] = df.apply(lambda row: fill_exp(row), axis=1)
    return df


def replace_words_with_count(df, column_name):
    df[column_name] = df[column_name].apply(lambda cell: len(cell.split(';')) if pd.notna(cell) else 0)
    return df


def fill_na_with_ratio(df):
    for column in df:
        # Drop NA values and get their counts
        if column == 'is_dev' or column == 'age_group' or column == 'worked_in_the_past' :
            continue
        non_na_values = df[column].dropna()
        value_counts = non_na_values.value_counts(normalize=True)
        # Fill NA values based on the ratio
        df[column] = df[column].apply(
            lambda x: np.random.choice(value_counts.index, p=value_counts.values) if pd.isna(x) else x
        )
    return df


def fill_is_dev_based_on_stack_experience(df, stack_experience_col, is_dev_col):
    # Fill is_dev based on stack_experience only if is_dev is NaN
    df[is_dev_col] = df.apply(
        lambda row: 'developer' if pd.isna(row[is_dev_col]) and row[stack_experience_col] > 0 else (
            'non_developer' if pd.isna(row[is_dev_col]) else row[is_dev_col]
        ), axis=1
    )
    return df


def fill_age_group_based_on_experience(df, experience_col, age_group_col):
    # Fill age_group based on experience only if age_group is NaN
    df[age_group_col] = df.apply(
        lambda row: 'old' if pd.isna(row[age_group_col]) and row[experience_col] > 20 else (
            "young" if pd.isna(row[age_group_col]) else row[age_group_col]
        ), axis=1
    )
    return df


def fill_worked_in_past(df, worked_col='worked_in_the_past', experience_col='years_of_experience'):
    # Check the conditions and update the 'worked in the past' column
    df[worked_col] = df.apply(
        lambda row: True if (pd.isna(row[worked_col]) and row[experience_col] > 0) or pd.isna(row[experience_col]) 
        else row[worked_col] if not pd.isna(row[worked_col])
        else False,
        axis=1
    )
    return df
    


train_df_copy = fill_b_and_experience(train_df_copy, 'B', 'years_of_experience')
train_df_copy = replace_words_with_count(train_df_copy, 'stack_experience')
train_df_copy = fill_na_with_ratio(train_df_copy)
train_df_copy = fill_is_dev_based_on_stack_experience(train_df_copy,'stack_experience','is_dev')
train_df_copy = fill_age_group_based_on_experience(train_df_copy,'years_of_experience','age_group')
train_df_copy = fill_worked_in_past(train_df_copy)

train_df_copy

### 4. **Categorical variables:**
In the columns containing categorical variables, we will convert the categorical values to numerical values. <br>
**Converting categorical values to numeric values may result in loss of information, and we did not want arbitrary numeric values to be obtained.** Therefore, in order to maintain the importance of the order of each categorical value, we built a function that will check for each categorical value the ratio between the times it appears in the same row with label 1 compared to the times it appears with label 0, so that the number we place in place of the categorical value is the same ratio , between 0 and 1. Thus, categorical values with a higher frequency of label '1' will receive a higher "value" in training and test datasets. <br>
**It is important to note that since the test file has no labels, and the function relies on the labels, we simply streamed the numerical values created for the train file into the test file.**

In [ ]:
def calculate_punishment_metric(train_df, label_column, feature_columns):
    if label_column not in train_df.columns:
        print(f"Error: '{label_column}' column does not exist in the training DataFrame.")
        return train_df, {}
    
    punishment_mappings = {}
    
    for feature in feature_columns:
        if feature not in train_df.columns:
            print(f"Feature column {feature} not found in the training DataFrame.")
            continue
        
        # Group by feature and count the total number of labels
        total_labels = train_df.groupby(feature)[label_column].count()
        
        # Group by feature and count the number of label 1 occurrences
        label_1_counts = train_df[train_df[label_column] == 1].groupby(feature)[label_column].count()
        
        # Combine the results into a single DataFrame
        comparison_df = pd.DataFrame({'Total Labels': total_labels, 'Label 1 Counts': label_1_counts})
        
        # Fill NaN values with 0 (in case some categories do not have label 1 occurrences)
        comparison_df = comparison_df.fillna(0)
        
        # Calculate the proportion of label 1 occurrences
        comparison_df['Proportion of Label 1'] = comparison_df['Label 1 Counts'] / comparison_df['Total Labels']
        
        # Calculate the punishment metric (Proportion)
        comparison_df['Punishment Metric'] = comparison_df['Proportion of Label 1']
        
        # Create a mapping from feature value to punishment metric
        punishment_mapping = comparison_df['Punishment Metric'].to_dict()
        punishment_mappings[feature] = punishment_mapping
        
        # Replace the values in the feature column with the punishment metric in train_df
        train_df[feature] = train_df[feature].map(punishment_mapping)
    
    return train_df, punishment_mappings

# Define the columns and apply the function to train_df
array1 = ['worked_in_the_past', 'age_group', 'disability', 'is_dev',
          'mental_issues', 'education', 'C', 'country', 'sex']

train_df_copy, punishment_mappings = calculate_punishment_metric(train_df_copy, 'label', array1)

def apply_punishment_metric(test_df, punishment_mappings):
    for feature, punishment_mapping in punishment_mappings.items():
        if feature in test_df.columns:
            test_df[feature] = test_df[feature].map(punishment_mapping).fillna(0)
        else:
            print(f"Feature column {feature} not found in the test DataFrame.")
    
    return test_df

### 5. **Normalization:**
It seems that the features "years_of_experience", "prev_salary", "A", "B", "D", "stack_experience" and "country" are not normalized (Their range of values is large). **We will normalize them so that all the data would be on the same scale, otherwise a bias may arise in the data.** For the purpose of normalization we will use the StandardScaler method.

In [ ]:
def scale_and_split(df, cols_to_scale, label_column=None, id_column=None):
    # Split the dataset into 'features' and 'label' if label_column is provided
    if label_column and label_column in df.columns:
        labels = df[label_column]
        df_features = df.drop(columns=[label_column])
    else:
        labels = None
        df_features = df.copy()
    
    # Drop the id_column if it is provided
    if id_column and id_column in df.columns:
        df_features = df_features.drop(columns=[id_column])
    
    # Prepare the ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[('scale', StandardScaler(), cols_to_scale)],
        remainder='passthrough'
    )
    
    # Apply the transformer to the data
    df_scaled_array = preprocessor.fit_transform(df_features)
    
    # Get the list of all column names in the transformed data
    all_cols = cols_to_scale + [col for col in df_features.columns if col not in cols_to_scale]
    
    # Convert the transformed array to a DataFrame
    df_scaled = pd.DataFrame(df_scaled_array, columns=all_cols)
    
    # Reorder the DataFrame to match the original column order
    df_scaled = df_scaled[df_features.columns]
    
    return df_scaled, labels

# Define the columns to scale
cols_to_scale = ['years_of_experience', 'prev_salary', 'A', 'B', 'D', 'stack_experience']

# Apply the function to the training DataFrame
train_df_copy, train_labels = scale_and_split(train_df_copy, cols_to_scale, label_column='label', id_column='ID')

# Display the scaled DataFrames and labels
train_df_copy

### 6. **Dimensionality:**
After creating the features of the technology types, we had a total amount of 133 features. **Such a large amount of features may cause overfitting,** so that the model will adapt itself too much to the training set. It means that the variance will be very high, the bias will be low, and the performance of the model on the 'test' set will be less good. Thats is why we will implement two ways to reduce the number of features:
- **Manual download** - in the beginning of the preprocessing stage, we dropped the "ID" and "label" columns, because they are not relevant for training the model. Also, since we created 116 new features of technology types, we selected only 20 technologies with the highest correlation to the label "1" for the purpose of training the models.
<br>
- **PCA** - before implementing it, we were debating which method to choose. We were afraid that using PCA will cost in losing information and geting lower peformance. On the other hand, we noticed that the "Feature Selection" method takes too much time to run the models. Finally, we decided to use the PCA method, with the main reason being due to runtime considerations of model training. In order to lose as little information as possible, we defined in the PCA implementation that the variance will keep 99% of the variance of the original dataset.

<br>
<br>
<br>
<br>
<br>

# **Part C - Running models**
In this part, we will train 4 selected models.
First, we will implement PCA and split the training dataset to "train" and "validation":

In [ ]:
## Check and drop 'ID' and 'label' columns if they exist
columns_to_drop = ['ID', 'label']
train_df_copy = train_df_copy.drop(
    columns=[col for col in columns_to_drop if col in train_df_copy.columns], inplace=False)

# Perform PCA for dimensionality reduction
pca = PCA(0.99)  # Adjust the number of components as needed
X_pca = pca.fit_transform(train_df_copy)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, train_df['label'], test_size=0.20, random_state=4242)

## Basic models

## 1. Logistic Regression model:

In [ ]:
# Initialize the Logistic Regression model
logistic_model = LogisticRegression()

# Define the parameter grid for GridSearchCV
param_grid = {
    'max_iter': [100],          # Maximum number of iterations
    'penalty': ['l1'],          # L1 regularization
    'C': [0.1],                 # Inverse of regularization strength
    'solver': ['saga'],         # Solver for handling L1 penalty
    'l1_ratio': [1]             # Only used if penalty is 'elasticnet'
}

# Set up GridSearchCV to find the best parameters using AUC as the scoring metric
optimizer_logistic = GridSearchCV(logistic_model, param_grid, scoring='roc_auc', cv=5, verbose=1, n_jobs=-1)

# Fit the model on the training data
optimizer_logistic.fit(X_train, y_train)

# Retrieve the best model found by GridSearchCV
best_logistic_model = optimizer_logistic.best_estimator_

# Predict probabilities on the test set for the positive class
y_prob = best_logistic_model.predict_proba(X_test)[:, 1]

# Calculate and print the AUC score
auc_score = roc_auc_score(y_test, y_prob)
print(f"Optimized AUC: {auc_score:.3f}")

# Plot the ROC curve for the best model
RocCurveDisplay.from_estimator(optimizer_logistic, X_test, y_test)

# Print the best parameters found by GridSearchCV
best_parameters = optimizer_logistic.best_params_
print("Best parameters found: ", best_parameters)

# Predict the class labels on the test set
y_pred = optimizer_logistic.predict(X_test)

# Compute and display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

## 2. Naive Bayes model:

In [ ]:
# Define the Gaussian Naive Bayes model
NB_model = GaussianNB()

# Define the parameter grid
param_grid = {
    'var_smoothing': [1e-7]  # Example values for var_smoothing
}

# Initialize GridSearchCV with more cross-validation folds and verbosity
optimizer_nb = GridSearchCV(NB_model, param_grid, scoring='roc_auc', cv=5, verbose=1, n_jobs=-1)

# Fit the optimizer to the training data
optimizer_nb.fit(X_train, y_train)

# Retrieve the best model found by GridSearchCV
best_nb_model = optimizer_nb.best_estimator_

# Predict probabilities and calculate AUC score
y_prob = best_nb_model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_prob)
print(f"Optimized AUC: {auc_score:.3f}")

# Plot ROC curve
RocCurveDisplay.from_estimator(optimizer_nb, X_test, y_test)
plt.show()

# Best parameters
best_parameters = optimizer_nb.best_params_
print("Best parameters found: ", best_parameters)

# Predict labels
y_pred = optimizer_nb.predict(X_test)

# Compute and display the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

## Advanced models

## 1. Random Forest model:

In [ ]:
# Define the random forest model
forest_model = RandomForestClassifier()

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100],           # Number of trees in the forest
    'max_depth': [20],               # Maximum depth of the tree
    'min_samples_split': [5],        # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1],         # Minimum number of samples required to be at a leaf node
    'bootstrap': [True],             # Whether bootstrap samples are used when building trees
    'max_features': [None],          # Number of features to consider when looking for the best split
    'class_weight': ['balanced', None]  # Weighting of classes for imbalance handling
}

# Set up GridSearchCV for hyperparameter tuning using AUC as the scoring metric
optimizer_forest = GridSearchCV(forest_model, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

# Fit the model on the training data
optimizer_forest.fit(X_train, y_train)

# Retrieve the best model found by GridSearchCV
best_forest_model = optimizer_forest.best_estimator_

# Predict probabilities on the test set for the positive class
y_prob = best_forest_model.predict_proba(X_test)[:, 1]

# Calculate and print the AUC score
auc_score = roc_auc_score(y_test, y_prob)
print(f"Optimized Random Forest AUC: {auc_score:.3f}")

# Plot the ROC curve for the best model
RocCurveDisplay.from_estimator(optimizer_forest, X_test, y_test)

# Print the best parameters found by GridSearchCV
best_parameters = optimizer_forest.best_params_
print("Best parameters found: ", best_parameters)

# Predict the class labels on the test set
y_pred = optimizer_forest.predict(X_test)

# Compute the confusion matrix to evaluate the performance of the classifier
conf_matrix = confusion_matrix(y_test, y_pred)

# Display the confusion matrix as a heatmap
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

## 2. Multi-Layer Perceptron model:

In [ ]:
# Define the Multi-Layer Perceptron (MLP) model
MLP_model = MLPClassifier()

# Expanded parameter grid for GridSearchCV to explore more hyperparameter combinations
param_grid = {
    'hidden_layer_sizes': [(100, 100)],  # Configuration of hidden layers
    'activation': ['tanh'],              # Activation function for the hidden layers
    'solver': ['sgd'],                   # Solver for weight optimization
    'alpha': [0.1],                      # L2 penalty (regularization term) parameter
    'learning_rate': ['adaptive'],       # Learning rate schedule for weight updates
    'learning_rate_init': [1],           # Initial learning rate
    'max_iter': [1000]                   # Maximum number of iterations
}

# Initialize GridSearchCV with the MLP model and the defined parameter grid
# Scoring is based on the AUC, using 5-fold cross-validation
optimizer_mlp = GridSearchCV(MLP_model, param_grid, scoring='roc_auc', cv=5, verbose=2, n_jobs=-1)

# Fit the optimizer to the training data to find the best hyperparameters
optimizer_mlp.fit(X_train, y_train)

# Retrieve the best model found by GridSearchCV
best_mlp_model = optimizer_mlp.best_estimator_

# Predict probabilities on the test set for the positive class
y_prob = best_mlp_model.predict_proba(X_test)[:, 1]

# Calculate and print the AUC score for the best model
auc_score = roc_auc_score(y_test, y_prob)
print(f"Multi-Layer Perceptron AUC: {auc_score:.3f}")

# Plot the ROC curve for the best model
RocCurveDisplay.from_estimator(optimizer_mlp, X_test, y_test)

# Print the best hyperparameters found by GridSearchCV
best_parameters = optimizer_mlp.best_params_
print("Best parameters found: ", best_parameters)

# Predict the class labels on the test set
y_pred = optimizer_mlp.predict(X_test)

# Compute the confusion matrix to evaluate the performance of the classifier
conf_matrix = confusion_matrix(y_test, y_pred)

# Display the confusion matrix as a heatmap
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

### It seems that the MLP is our best model!
So we will analyze its performance and later on run it with the Test dataset.

## Feature Importance
We want to know which features are mostly contribute the model's performance. <br>
Since we learned in class only "Feature Importance" method which is used only for Random Forest model, we will use a technique called **"Permutation Feature Importance"**. This method helps to measure how much the model’s performance **decreases** when the values of a particular feature are randomly shuffled or permuted while keeping other variables unchanged.

In [ ]:
# Calculate permutation feature importance
result = permutation_importance(best_mlp_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

# Get feature importances and their corresponding indices
importance_means = result.importances_mean
importance_std = result.importances_std
indices = np.argsort(importance_means)[::-1]

# Get PCA components and original feature names
pca_components = pca.components_
original_feature_names = train_df_copy.columns

# Calculate feature importance based on PCA loadings
feature_importance = np.abs(pca_components.T @ importance_means)
sorted_indices = np.argsort(feature_importance)[::-1]

# Plot feature importances with original feature names
plt.figure(figsize=(12, 8))
plt.title("Feature Importance using Permutation")
plt.bar(range(len(original_feature_names)), feature_importance[sorted_indices], color="b", align="center")
plt.xticks(range(len(original_feature_names)), [original_feature_names[i] for i in sorted_indices], rotation=90)
plt.xlim([-1, len(original_feature_names)])
plt.xlabel("Features")
plt.ylabel("Importance")
plt.show()

As you can see, the most important feature is "stack_experience", and right after it come the different technologies. <br> <br>
**This plot supports our assumption that knowing different technologies and programming languages increases a person's chances to get hired for a job.**

<br>
<br>
<br>
<br>
<br>

# **Part D - Model Evaluating**

### Here we can see again the Confusion Matrix of the MLP model:

In [ ]:
# Plot the ROC curve for the best MLP model found by GridSearchCV
RocCurveDisplay.from_estimator(optimizer_mlp, X_test, y_test)

# Retrieve and print the best hyperparameters found by GridSearchCV
best_parameters = optimizer_mlp.best_params_
print("Best parameters found: ", best_parameters)

# Predict the class labels on the test set using the best MLP model
y_pred = optimizer_mlp.predict(X_test)

# Compute the confusion matrix to evaluate the performance of the model
conf_matrix = confusion_matrix(y_test, y_pred)

# Display the confusion matrix as a heatmap for visual inspection
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

Here is a breakdown of each cell in the matrix:<br>
- **TN** (True Negatives, upper left corner, 4614): The model correctly predicted 0 (negative classification) when the actual class was 0. <br>
- **FP** (False Positives, upper right corner, 497): The model falsely predicted 1 (positive classification) when the actual class was 0.<br>
- **FN** (False Negatives, lower left corner, 437): The model falsely predicted 0 (negative classification) when the actual class was 1.<br>
- **TP** (True Positives, lower right corner, 5545): The model correctly predicted 1 (positive classification) when the actual class was 1.<br>
<br> From these values we can calculate several important performance indicators:

1. **Accuracy:** <br>
(TP+TN) / TP+TN+FP+FN = (5554+4604) / 5554+4604+507+428 ≈ 0.916 <br>
It means that in 91.6% of the cases, the model correctly predicts the classifications.

2. **Precision:** <br>
TP / (TP+FP) = 5554 / (5545+507) ≈ 0.916 <br>
It means that when the model predicts a positive classification, it is correct about 91.6% of the time, indicating a low rate of false positive predictions.

3. **Sensitivity:** <br>
TP / (TP+FN) = 5545 / (5545+428) ≈ 0.928 <br>
It means that the model correctly identifies 92.8% of the actual positive cases, which indicates a low rate of false negative predictions.

4. **Specificity:** <br>
TN / (TN+FP) = 4604 / (4604+507) ≈ 0.90 <br>
It means that the model correctly identifies 90% of the actual negative cases, indicating good performance in preventing false positive predictions.

### Here we will run our MLP model without using K-Cross validation:

In [ ]:
MLP_model = MLPClassifier(
    hidden_layer_sizes=(100, 100),
    activation='tanh',
    solver='sgd',
    alpha=0.1,
    learning_rate='adaptive',
    learning_rate_init=0.1,
    max_iter=1000
)

# Fit the model to the training data
MLP_model.fit(train_df_copy, train_df['label'])

# Display the ROC curve
RocCurveDisplay.from_estimator(MLP_model, train_df_copy, train_df['label'])
plt.title("ROC Curve")
plt.show()

# Predict the test data
y_pred = MLP_model.predict(train_df_copy)

# Compute and display the confusion matrix
conf_matrix = confusion_matrix(train_df['label'], y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Calculate and print the AUC score
y_proba = MLP_model.predict_proba(train_df_copy)[:, 1]
auc_score = roc_auc_score(train_df['label'], y_proba)
print(f"AUC score: {auc_score:.3f}")

As you can see, the model's AUC without K-Cross validation is only 0.001 higher than the model's AUC using K-Cross validation. <br>
**It indicates that the model is not overfitted.** <br>
It would have been considered as overfitted **if the AUC was much lower after using K-Cross validation**. <Br>
After all, we expect to get high AUC without using K-Cross validation, so getting almost the same results after testing on the validation set is a good sign.

## Here you can see all the 4 ROC curves of the model above:

In [ ]:
# Dictionary to store the different models and their corresponding optimized versions
models = {
    "Logistic Regression": optimizer_logistic,
    "Naive Bayes": optimizer_nb,
    "Random Forest": optimizer_forest,
    "MLP": optimizer_mlp
}

# Set up the plot for ROC curves for each model
plt.figure(figsize=(10, 8))

# Iterate over each model in the dictionary to calculate and plot the ROC curve
for name, model in models.items():
    y_proba = model.predict_proba(X_test)[:, 1]  # Get the predicted probabilities for the positive class
    fpr, tpr, _ = roc_curve(y_test, y_proba)     # Compute the false positive rate and true positive rate
    auc_score = roc_auc_score(y_test, y_proba)   # Calculate the AUC score for the ROC curve
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc_score:.3f})')  # Plot the ROC curve for this model

# Add labels and title to the ROC plot
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')  # Add a legend to identify each model's ROC curve
plt.show()  # Display the plot

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>



# **Part E - Prediction**

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,FunctionTransformer, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_curve, RocCurveDisplay, accuracy_score, roc_auc_score, confusion_matrix, classification_report,ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.inspection import permutation_importance
import os


### First, we will create a function that units all the functions from the preprocessing stage:

In [ ]:
# Unit all the functions from the preprocessing stage and apply them on the "train" and "test" datasets.

def preprocess_all(train_df, test_df):
    
    def cap_outliers_percentiles_multi(df, columns, lower_percentile=0.01, upper_percentile=0.99):
        for column in columns:
            lower_bound = df[column].quantile(lower_percentile)
            upper_bound = df[column].quantile(upper_percentile)
            df[column] = df[column].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))
        return df
    numeric_columns = ['A', 'B', 'D']
    
    train_df = cap_outliers_percentiles_multi(train_df, numeric_columns)
    
    def process_stack_experience(train_df, label_column, top_n=20):
        technologies = train_df['stack_experience'].dropna().str.split(';').tolist()
        flattened_technologies = [tech for sublist in technologies for tech in sublist]
        technology_counts = Counter(flattened_technologies)
        unique_technologies_count = len(technology_counts)
        stack_experience_dummies = train_df['stack_experience'].str.get_dummies(sep=';')
        correlation_with_label = stack_experience_dummies.corrwith(train_df[label_column]).sort_values(ascending=False)
        top_technologies = correlation_with_label.head(top_n).index
        train_df = pd.concat([train_df, stack_experience_dummies[top_technologies]], axis=1)
        return train_df, unique_technologies_count, top_technologies, correlation_with_label
    
    train_df, unique_technologies_count, top_technologies, correlation_with_label = process_stack_experience(train_df, 'label')
    
    def add_top_technologies_to_test_df(test_df, top_technologies):
        stack_experience_dummies_test = test_df['stack_experience'].str.get_dummies(sep=';')
        test_top_tech_dummies = stack_experience_dummies_test.reindex(columns=top_technologies, fill_value=0)
        test_df = pd.concat([test_df, test_top_tech_dummies], axis=1)
        return test_df
    
    test_df = add_top_technologies_to_test_df(test_df, top_technologies)
        
    
    def fill_b_and_experience(df, b_col, exp_col):
        b_mean = df[b_col].mean()
        exp_mean = df[exp_col].mean()

        def fill_b(row):
            if pd.isna(row[b_col]):
                if pd.isna(row[exp_col]):
                    return b_mean
                else:
                    return max(row[exp_col] - 5, 0)
            else:
                return row[b_col]

        def fill_exp(row):
            if pd.isna(row[exp_col]):
                if pd.isna(row[b_col]):
                    return exp_mean
                else:
                    return row[b_col] + 5
            else:
                return row[exp_col]

        df[b_col] = df.apply(lambda row: fill_b(row), axis=1)
        df[exp_col] = df.apply(lambda row: fill_exp(row), axis=1)
        return df


    def replace_words_with_count(df, column_name):
        df[column_name] = df[column_name].apply(lambda cell: len(cell.split(';')) if pd.notna(cell) else 0)
        return df


    def fill_na_with_ratio(df):
        for column in df:
            if column == 'is_dev' or column == 'age_group' or column == 'worked_in_the_past' :
                continue
            non_na_values = df[column].dropna()
            value_counts = non_na_values.value_counts(normalize=True)
            df[column] = df[column].apply(
                lambda x: np.random.choice(value_counts.index, p=value_counts.values) if pd.isna(x) else x
            )
        return df


    def fill_is_dev_based_on_stack_experience(df, stack_experience_col, is_dev_col):
        df[is_dev_col] = df.apply(
            lambda row: 'developer' if pd.isna(row[is_dev_col]) and row[stack_experience_col] > 0 else (
                'non_developer' if pd.isna(row[is_dev_col]) else row[is_dev_col]
            ), axis=1
        )
        return df


    def fill_age_group_based_on_experience(df, experience_col, age_group_col):
        df[age_group_col] = df.apply(
            lambda row: 'old' if pd.isna(row[age_group_col]) and row[experience_col] > 20 else (
                "young" if pd.isna(row[age_group_col]) else row[age_group_col]
            ), axis=1
        )
        return df


    def fill_worked_in_past(df, worked_col='worked_in_the_past', experience_col='years_of_experience'):
        df[worked_col] = df.apply(
            lambda row: True if (pd.isna(row[worked_col]) and row[experience_col] > 0) or pd.isna(row[experience_col]) 
            else row[worked_col] if not pd.isna(row[worked_col])
            else False,
            axis=1
        )
        return df
        
        
    train_df = fill_b_and_experience(train_df, 'B', 'years_of_experience')
    test_df = fill_b_and_experience(test_df, 'B', 'years_of_experience')
    train_df = replace_words_with_count(train_df, 'stack_experience')
    test_df = replace_words_with_count(test_df, 'stack_experience')
    train_df = fill_na_with_ratio(train_df)
    test_df = fill_na_with_ratio(test_df)
    train_df = fill_is_dev_based_on_stack_experience(train_df,'stack_experience','is_dev')
    test_df = fill_is_dev_based_on_stack_experience(test_df,'stack_experience','is_dev')
    train_df = fill_age_group_based_on_experience(train_df,'years_of_experience','age_group')
    test_df = fill_age_group_based_on_experience(test_df,'years_of_experience','age_group')
    train_df = fill_worked_in_past(train_df)
    test_df = fill_worked_in_past(test_df)
    
    def calculate_punishment_metric(train_df, label_column, feature_columns):
        if label_column not in train_df.columns:
            print(f"Error: '{label_column}' column does not exist in the training DataFrame.")
            return train_df, {}
        punishment_mappings = {}
        for feature in feature_columns:
            if feature not in train_df.columns:
                print(f"Feature column {feature} not found in the training DataFrame.")
                continue
            total_labels = train_df.groupby(feature)[label_column].count()
            label_1_counts = train_df[train_df[label_column] == 1].groupby(feature)[label_column].count()
            comparison_df = pd.DataFrame({'Total Labels': total_labels, 'Label 1 Counts': label_1_counts})
            comparison_df = comparison_df.fillna(0)
            comparison_df['Proportion of Label 1'] = comparison_df['Label 1 Counts'] / comparison_df['Total Labels']
            comparison_df['Punishment Metric'] = comparison_df['Proportion of Label 1']
            punishment_mapping = comparison_df['Punishment Metric'].to_dict()
            punishment_mappings[feature] = punishment_mapping
            train_df[feature] = train_df[feature].map(punishment_mapping)
        return train_df, punishment_mappings
    
    array1 = ['worked_in_the_past', 'age_group', 'disability', 'is_dev',
          'mental_issues', 'education', 'C', 'country', 'sex']
    train_df, punishment_mappings = calculate_punishment_metric(train_df, 'label', array1)
    
    def apply_punishment_metric(test_df, punishment_mappings):
        for feature, punishment_mapping in punishment_mappings.items():
            if feature in test_df.columns:
                test_df[feature] = test_df[feature].map(punishment_mapping).fillna(0)
            else:
                print(f"Feature column {feature} not found in the test DataFrame.")

        return test_df

    test_df = apply_punishment_metric(test_df, punishment_mappings)
    
    def scale_and_split(df, cols_to_scale, label_column=None, id_column=None):
        if label_column and label_column in df.columns:
            labels = df[label_column]
            df_features = df.drop(columns=[label_column])
        else:
            labels = None
            df_features = df.copy()
        if id_column and id_column in df.columns:
            df_features = df_features.drop(columns=[id_column])
        preprocessor = ColumnTransformer(
            transformers=[('scale', StandardScaler(), cols_to_scale)],
            remainder='passthrough'
        )
        df_scaled_array = preprocessor.fit_transform(df_features)
        all_cols = cols_to_scale + [col for col in df_features.columns if col not in cols_to_scale]
        df_scaled = pd.DataFrame(df_scaled_array, columns=all_cols)
        df_scaled = df_scaled[df_features.columns]
        return df_scaled, labels
    
    cols_to_scale = ['years_of_experience', 'prev_salary', 'A', 'B', 'D', 'stack_experience']
    train_df, train_labels = scale_and_split(train_df, cols_to_scale, label_column='label', id_column='ID')
    test_df, _ = scale_and_split(test_df, cols_to_scale, label_column='label', id_column='ID')
        
    return train_df, test_df

# Pipline
Here, you can run the important things:
1. Loading the datasets and applying the preprocessing stage.
2. Defining our best model and training it.
3. Creating predictions on the data from the "test" dataset, and save it in a file called "result_11.csv"

In [ ]:
# Loading the datasets, doing preprocessing on the datasets, training the best model and create predictions.

def pipeline(train_csv, test_csv, results_11_csv):
    # Load the training and test datasets from CSV files
    train_df = pd.read_csv(train_csv)
    test_df = pd.read_csv(test_csv)
    
    # Extract labels from the training data and IDs from the test data
    train_labels = train_df.label
    test_ID = test_df.ID
    
    # Preprocess the data (assumes preprocess_all is a defined function)
    train_df_run, test_df_run = preprocess_all(train_df, test_df)
    
    # Set up the training and test datasets for the model
    X_train = train_df_run
    y_train = train_labels
    X_test = test_df_run.drop(columns=['ID'], errors='ignore')  # Drop ID column from test set if it exists
    
    # Define and train the MLP model with specified parameters
    mlp_model = MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=(100, 100), 
                              learning_rate='adaptive', learning_rate_init=0.1, solver='sgd')
    mlp_model.fit(X_train, y_train)
    
    # Predict probabilities for the test set
    probabilities = mlp_model.predict_proba(X_test)
    
    # Extract probabilities for the positive class (class 1)
    positive_class_probabilities = probabilities[:, 1]
    
    # Remove the existing results file if it exists to avoid overwriting issues
    if os.path.exists(results_11_csv):
        os.remove(results_11_csv)
    
    # Create a DataFrame with test IDs and predicted probabilities
    prediction_file = pd.DataFrame({'ID': test_ID, 'probability': positive_class_probabilities})
    
    # Save the predictions to a CSV file
    prediction_file.to_csv(results_11_csv, index=False)
    
    # Print confirmation that the results file has been created
    print(f"The file '{results_11_csv}' is created and ready to use!")

# Run the pipeline function with specified CSV files
pipeline('train.csv', 'test.csv', 'results_11.csv')